In [ ]:
import pandas as pd

In [ ]:
customer_df=pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv')
geolocation_df=pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv')
order_items_df=pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv')
order_payments_df=pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv')
order_review_df=pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv', parse_dates=[
    'review_creation_date', 'review_answer_timestamp'
])
order_df=pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv', parse_dates=[
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
])
product_df=pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv')
seller_df=pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv')
product_name_df=pd.read_csv('/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv')

In [ ]:
modified_order_df=order_df.merge(order_items_df,how='inner',on='order_id')
modified_order_df=modified_order_df.merge(order_payments_df,how='inner',on='order_id')
modified_order_df=modified_order_df.merge(order_review_df,how='inner',on='order_id')
modified_order_df=modified_order_df.merge(product_df,how='inner',on='product_id')
modified_order_df=modified_order_df.merge(seller_df,how='inner',on='seller_id')
modified_order_df=modified_order_df.merge(product_name_df,how='inner',on='product_category_name')
modified_order_df=modified_order_df.merge(customer_df,how='inner',on='customer_id')

In [ ]:
print(modified_order_df.info())

In [ ]:
print(modified_order_df.isnull().sum().sort_values(ascending=False))

In [ ]:
olist_df = modified_order_df.drop(['review_comment_title', 'review_comment_message'], axis=1)
print(olist_df.sample(7))

In [ ]:
delivery_durations = (
    modified_order_df['order_delivered_customer_date'] - modified_order_df['order_purchase_timestamp']
).dt.days
avg_delivery_days = round(delivery_durations.dropna().mean())

In [ ]:
mask_missing_customer = modified_order_df['order_delivered_customer_date'].isna()
modified_order_df.loc[mask_missing_customer, 'order_delivered_customer_date'] = (
    modified_order_df.loc[mask_missing_customer, 'order_purchase_timestamp'] +
    pd.to_timedelta(avg_delivery_days, unit='D')
)

In [ ]:
carrier_durations = (
    modified_order_df['order_delivered_carrier_date'] - modified_order_df['order_purchase_timestamp']
).dt.days
avg_carrier_days = round(carrier_durations.dropna().mean())

mask_missing_carrier = modified_order_df['order_delivered_carrier_date'].isna()
modified_order_df.loc[mask_missing_carrier, 'order_delivered_carrier_date'] = (
    modified_order_df.loc[mask_missing_carrier, 'order_purchase_timestamp'] +
    pd.to_timedelta(avg_carrier_days, unit='D')
)

In [ ]:
# drop review text columns with many nulls
modified_order_df = modified_order_df.drop(['review_comment_title', 'review_comment_message'], axis=1)

In [ ]:

print(modified_order_df.isnull().sum().sort_values(ascending=False))

In [ ]:
approval_durations = (
    modified_order_df['order_approved_at'] - modified_order_df['order_purchase_timestamp']
).dt.total_seconds() / 3600  # convert to hours

# Compute average approval time
avg_approval_hours = round(approval_durations.dropna().mean())

In [ ]:
mask_missing_approval = modified_order_df['order_approved_at'].isna()
modified_order_df.loc[mask_missing_approval, 'order_approved_at'] = (
    modified_order_df.loc[mask_missing_approval, 'order_purchase_timestamp'] +
    pd.to_timedelta(avg_approval_hours, unit='h')
)

In [ ]:
print(modified_order_df.isnull().sum().sort_values(ascending=False))

In [ ]:
modified_order_df.to_csv('Final_olist_dataset.csv', index=False)

In [ ]:
modified_order_df.sample(10)

In [ ]:
modified_order_df['customer_city']

In [ ]:
modified_order_df.drop(['customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state'], axis=1, inplace=True)

In [ ]:
modified_order_df.columns

In [ ]:
customer_df_ind = pd.read_csv('/kaggle/input/customer-data-ind/customer_data(indian).csv')

modified_order_df=modified_order_df.merge(customer_df_ind,how='inner',on='customer_id')

In [ ]:
modified_order_df.info()

In [ ]:
modified_order_df.isnull().sum()

In [ ]:
modified_order_df.sample(10)

In [ ]:
modified_order_df.shape

In [ ]:
modified_order_df.columns

In [ ]:
modified_order_df['product_description_lenght']

In [ ]:
modified_order_df.drop(['product_length_cm', 'product_height_cm', 'product_description_lenght', 'product_width_cm'], axis=1, inplace=True)

In [ ]:
modified_order_df.to_csv('datset_ecommerce_s.csv', index=False)

In [ ]:
print(modified_order_df['seller_id'].nunique())